In [ ]:
using Luna
using Optim
using FFTW
using DelimitedFiles
using Printf
import Luna: Hankel 
using DSP

In [ ]:
using PyPlot
pygui(true)

true

# Pulse Compression
Compressing the output pulse by finding the best GDD value that maximizes the pulse intensity/power (?)

In [ ]:
main_folder = joinpath("output_sorted", "complex spectrum output dummy data", "run_2025_01_23__12_02_16")
#main_folder = joinpath("output_sorted", "complex spectrum output dummy data")

"output_sorted\\complex spectrum output dummy data\\run_2025_01_23__12_02_16"

In [ ]:
# omega_path = joinpath(main_folder, "omega_axis.txt")
# time_path = joinpath(main_folder, "time_axis.txt")
spectrum_path = joinpath(main_folder, "complex_spectrum_output.txt")

# ω = readdlm(omega_path, Float64)[:,1]                                         # angular frequencies
# t = readdlm(time_path, Float64)[:,1]                                         # time steps 
data = readdlm(spectrum_path, '\t') 

ω = data[:, 1]
Ẽω_real = data[:, 2]
Ẽω_imag = data[:, 3]
Ẽω = Ẽω_real .+ 1im .*Ẽω_imag

println(typeof(Ẽω))

Vector{ComplexF64}


In [ ]:
# simulation grid parameters
L = 380.1934e-3          # MPC cell length [m]
λ0 = 1030e-9        # central wavelength [m]
λlims = (700e-9, 1400e-9) #(700e-9, 1400e-9) #(600e-9, 1500e-9) # wavelength range of interest [m]
trange = 10*150e-15 #300.0e-15 #0.05e-12    # total extent of time window required [s], default was 50fs (NOTE: if this is too short, the range is extended automatically; this value was taken from an older version of the code; no justification!)


grid = Grid.RealGrid(L, λ0, λlims, trange)               # set up time & space grid for gradient approximation

┌ Info: Freq limits 0.21 - 0.43 PHz
└ @ Luna.Grid C:\Users\muelderk\.julia\packages\Luna\7FAkO\src\Grid.jl:40
┌ Info: Samples needed: 3854.47, samples: 4096, δt = 389.16 as
└ @ Luna.Grid C:\Users\muelderk\.julia\packages\Luna\7FAkO\src\Grid.jl:44
┌ Info: Requested time window: 1500.0 fs, actual time window: 1594.0 fs
└ @ Luna.Grid C:\Users\muelderk\.julia\packages\Luna\7FAkO\src\Grid.jl:46
┌ Info: Grid: samples 2048 / 4096, ωmax 4.04e+15 / 8.07e+15
└ @ Luna.Grid C:\Users\muelderk\.julia\packages\Luna\7FAkO\src\Grid.jl:78


Luna.Grid.RealGrid(0.3801934, 1.03e-6, [-7.96995811460118e-13, -7.96217495237989e-13, -7.954391790158599e-13, -7.94660862793731e-13, -7.938825465716019e-13, -7.931042303494729e-13, -7.923259141273438e-13, -7.915475979052148e-13, -7.907692816830859e-13, -7.899909654609568e-13  …  7.892126492388278e-13, 7.899909654609568e-13, 7.907692816830859e-13, 7.915475979052148e-13, 7.923259141273438e-13, 7.931042303494729e-13, 7.938825465716019e-13, 7.94660862793731e-13, 7.954391790158599e-13, 7.96217495237989e-13], [0.0, 3.941793179357254e12, 7.883586358714508e12, 1.1825379538071762e13, 1.5767172717429016e13, 1.970896589678627e13, 2.3650759076143523e13, 2.7592552255500777e13, 3.153434543485803e13, 3.547613861421528e13  …  4.0009200770476125e15, 4.00486187022697e15, 4.008803663406327e15, 4.0127454565856845e15, 4.0166872497650415e15, 4.020629042944399e15, 4.024570836123756e15, 4.0285126293031135e15, 4.0324544224824705e15, 4.036396215661828e15], [-7.96995811460118e-13, -7.966066533490535e-13, -7.9621

In [ ]:
Ẽω_comp = GDD -> -maximum(FFTW.irfft(Fields.prop_taylor(Ẽω, grid, [0, 0, GDD*1e-30], λ0), length(grid.t), 1))
#println(typeof(Ẽω_comp))

result = optimize(Ẽω_comp, 0, 1000)
best_gdd = Optim.minimizer(result)

ϕ_comp = [0, 0, best_gdd*1e-30, 0]
Ẽω_comp = Fields.prop_taylor(Ẽω, grid, ϕ_comp, λ0)

# for i in [100, 1000, 10, 1]
#     result = optimize(Ẽω_comp, i)                # find bestgdd value from different starting points
#     gdd = Optim.minimizer(result)
#     if Ẽω_comp(bestgdd) > Ẽω_comp(gdd)
#         bestgdd = gdd
#     end
# end

1025-element Vector{ComplexF64}:
   2493.431865349321 - 856.3701248801377im
  1316.0831050426873 + 2245.8647120039795im
  -1860.935587239619 + 1676.126718704411im
 -1887.8465493821727 - 1390.9354240364587im
   906.3639719783575 - 1929.6598232681056im
  1814.3376464736104 + 482.206794348455im
  -181.1010393695233 + 1589.331765171006im
 -1328.7265604708703 - 36.95057088869453im
  43.243123887338584 - 1117.2502947265516im
  1028.9921226254799 + 150.03031235811375im
                     ⋮
  -344.3287563971525 + 80.5952191527171im
  -295.8249588949147 + 161.57781175095096im
 -229.47117502599625 + 231.03392065551867im
 -147.95527398449852 + 283.2571512336242im
  -55.70667360736914 + 313.1896049196607im
   41.18324901512851 + 317.00580837181224im
  135.24724271905131 + 292.70801939986626im
  218.15530369356742 + 240.66202206645468im
   281.4506323751915 + 163.98801372471814im

In [ ]:
# Add post compression of the pulse (remove GDD)

    
# find the best GDD value for which pulse power/intensity? is maximized
fun = zz -> -maximum((abs((FFTW.irfft((Fields.prop_taylor(Ẽω, grid, [0, 0, zz*1e-30], λ0), length(grid.t), 1)))).^2)) # returns negative maximum intensity for pulse with a certain GDD
result = optimize(fun, 0, 1000)
best_gdd = Optim.minimizer(result)                        
for initial_gdd = [100,1000,10,1]
    result = optimize(fun, initial_gdd)                # find bestgdd value from different starting points
    gdd = Optim.minimizer(result)
    if fun(bestgdd) > fun(gdd)
        bestgdd = gdd
    end
end

# GDD_comp = -600e-30
ϕ_comp = [0, 0, best_gdd*1e-30, 0]
Ẽω_comp = Fields.prop_taylor(Ẽω, grid, ϕ_comp, λ0)

MethodError: MethodError: no method matching irfft(::Tuple{Vector{ComplexF64}, Int64, Int64})

Closest candidates are:
  irfft(!Matched::AbstractArray{<:Complex{<:Union{Integer, Rational}}}, !Matched::Integer, !Matched::Any)
   @ AbstractFFTs C:\Users\muelderk\.julia\packages\AbstractFFTs\4iQz5\src\definitions.jl:314
  irfft(!Matched::AbstractArray{<:Real}, !Matched::Integer, !Matched::Any)
   @ AbstractFFTs C:\Users\muelderk\.julia\packages\AbstractFFTs\4iQz5\src\definitions.jl:313
  irfft(!Matched::AbstractArray, !Matched::Integer, !Matched::Any)
   @ AbstractFFTs C:\Users\muelderk\.julia\packages\AbstractFFTs\4iQz5\src\definitions.jl:306
  ...
